# Загрузка состава индекса IMOEX

In [1]:
import requests
import pandas as pd
import cvxpy as cp
import numpy as np

In [2]:
print('cvxpy', cp.__version__)
print('numpy', np.__version__)
print('pandas', pd.__version__)

cvxpy 1.0.31
numpy 1.19.1
pandas 1.1.0


Загрузка весов инструментов в индексе

In [3]:
analytics_url = \
'http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/IMOEX.json?start={i}&analytics.columns={columns}'

columns = ['ticker', 'weight']

res = requests.get(analytics_url.format(i=0, columns=','.join(columns)))
res_json = res.json()

pagesize_index = res_json['analytics.cursor']['columns'].index('PAGESIZE')
pagesize = res_json['analytics.cursor']['data'][0][pagesize_index]

total_index = res_json['analytics.cursor']['columns'].index('TOTAL')
total = res_json['analytics.cursor']['data'][0][total_index]

In [4]:
analitics_rows = []
for i in range(0, total, pagesize):
    res = requests.get(analytics_url.format(i=i, columns=','.join(columns)))
    res_json = res.json()
    analitics_rows += res_json['analytics']['data']

In [5]:
analytics_data = pd.DataFrame(analitics_rows, columns=columns)
analytics_data.head()

,ticker,weight
0,AFKS,0.51
1,AFLT,0.29
2,ALRS,1.37
3,CBOM,0.28
4,CHMF,1.24


Загрузка актуальных цен закрытия и размеров лотов

In [6]:
securities_metadata_url = \
'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/metadata.json'

securities_url = \
'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json?securities={securities}'

res = requests.get(securities_metadata_url)
res_json = res.json()

lotsize_index = res_json['securities']['columns'].index('LOTSIZE')
lastprice_index = res_json['marketdata']['columns'].index('LAST')

In [7]:
lotsize_rows = []
lastprice_rows = []
for i in range(0, total, 10):
    securities = ','.join(analytics_data.ticker[i:i+10])
    res = requests.get(securities_url.format(securities=securities))
    res_json = res.json()
    for securities_row in res_json['securities']['data']:
        lotsize_rows.append(securities_row[lotsize_index])
    for marketdata_row in res_json['marketdata']['data']:
        lastprice_rows.append(marketdata_row[lastprice_index])

Состав индекса

In [8]:
analytics_data['lotsize'] = lotsize_rows
analytics_data['lastprice'] = lastprice_rows
analytics_data['weight'] = analytics_data['weight'] * 0.01
analytics_data

,ticker,weight,lotsize,lastprice
0,AFKS,0.0051,100,19.98500
1,AFLT,0.0029,10,82.20000
2,ALRS,0.0137,10,69.98000
3,CBOM,0.0028,100,5.91900
4,CHMF,0.0124,1,938.00000
5,DSKY,0.0039,10,120.78000
6,FEES,0.0037,10000,0.20030
7,FIVE,0.0248,1,2825.50000
8,GAZP,0.1299,10,188.80000
9,GMKN,0.0718,1,20786.00000


# Формирование портфеля

Входные данные:

* $T$ - объем инвестиций в рублях
* $a$ - вектор весов инструментов в индексе
* $p$ - вектор актуальных цен закрытия
* $l$ - вектор количества бумаг в одном лоте (размеры лотов)
* $n$ - количество инструментов в индексе

Требуется найти:

* $x$ - вектор весов инструментов в портфеле
* $y$ - вектор количества лотов

Решаем задачу оптимизации с ограничениями

$$f(x) = \sum_i^n (x_i - a_i)^2 \to \min$$

$$\begin{cases}
\sum_i^n x_i \leq 1 \\
\forall i: x_i \geq 0 \\
\forall i: y_i = T x_i\frac{1}{p_i} \frac{1}{l_i}\\
\forall i: y_i \in \mathbb Z
\end{cases}$$

In [9]:
total = 1000000
weight = analytics_data.weight.values
price = analytics_data.lastprice.values
lotsize = analytics_data.lotsize.values

x_weight = cp.Variable(analytics_data.shape[0], nonneg=True)
y_count = cp.Variable(analytics_data.shape[0], integer=True)

obj = cp.Minimize(cp.sum_squares(x_weight - weight))
constr1 = (y_count == total * x_weight / price / lotsize)
constr2 = (cp.sum(x_weight) <= 1)

problem = cp.Problem(obj, [constr1, constr2])
problem.solve(solver=cp.ECOS_BB)
problem.status

'optimal_inaccurate'

Состав портфеля

In [10]:
portfolio = pd.DataFrame(analytics_data[['ticker']])
portfolio['weight'] = x_weight.value.round(4)
portfolio['count'] = y_count.value * analytics_data.lotsize
portfolio['total'] = y_count.value * analytics_data.lotsize * analytics_data.lastprice
portfolio

,ticker,weight,count,total
0,AFKS,0.0060,300.0,5995.50
1,AFLT,0.0033,40.0,3288.00
2,ALRS,0.0140,200.0,13996.00
3,CBOM,0.0030,500.0,2959.50
4,CHMF,0.0122,13.0,12194.00
5,DSKY,0.0036,30.0,3623.40
6,FEES,0.0040,20000.0,4006.00
7,FIVE,0.0254,9.0,25429.50
8,GAZP,0.1303,690.0,130272.00
9,GMKN,0.0624,3.0,62358.00


Стоимость портфеля

In [11]:
round(portfolio.total.sum(), 2)

983744.45